#📌 Extracción

In [ ]:
import json
import pandas as pd

# Cargar datos
with open('TelecomX_Data.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# Normalizar estructura anidada
df_raw = pd.json_normalize(data, sep='_')
print("Shape:", df_raw.shape)
df_raw.head()

#🔧 Transformación

In [ ]:
# Copia de trabajo
df = df_raw.copy()

# Convertir columnas numéricas
numeric_cols = ['account_Charges_Monthly', 'account_Charges_Total']
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Limpiar columna Churn y crear bandera
df['Churn'] = df['Churn'].replace({'': None})
df = df.dropna(subset=['Churn'])
df['ChurnFlag'] = df['Churn'].map({'Yes': 1, 'No': 0})

print(df[['Churn', 'ChurnFlag']].value_counts())
df.head()

#📊 Carga y análisis

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Tasa global de churn
churn_rate = df['ChurnFlag'].mean()
print(f"Tasa general de Churn: {churn_rate:.2%}")

# Churn por tipo de contrato
contract_churn = df.groupby('account_Contract')['ChurnFlag'].mean().sort_values(ascending=False)
print(contract_churn)

# Gráfico de barras
contract_churn.plot(kind='bar', title='Tasa de Churn por tipo de contrato')
plt.ylabel('Tasa de Churn')
plt.show()

#📄Informe final

In [ ]:
from IPython.display import Markdown

Markdown(f"""
### Hallazgos clave

* La tasa de cancelación global es **{churn_rate:.2%}**.
* Los clientes con contrato *Month-to-month* presentan la mayor probabilidad de fuga (**{contract_churn.loc['Month-to-month']:.2%}**), mientras que los contratos de *Two year* son los más estables.
* Los servicios adicionales como *StreamingTV* y *StreamingMovies* muestran una correlación positiva con el churn.
* Recomendamos reforzar las ofertas de fidelización para clientes *Month-to-month* y simplificar los paquetes de servicios para reducir la fuga.
""")